In [1]:
# inportar librerias
import sqlite3  # Módulo para trabajar con bases de datos SQLite3
import pandas as pd  # Módulo para trabajar con estructuras de datos tipo DataFrame

# conectarse a la base de datos
conn = sqlite3.connect("data/db_movies")
cur = conn.cursor()  # Crear un cursor para ejecutar sentencias SQL

# consultamos las tablas de la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',), ('movies_clean',), ('full_table',), ('data_clean',)]


In [2]:
# ver tabla de data_clean
df = pd.read_sql_query("SELECT * FROM data_clean", conn)
df.head(10)

,movieId,title,year,userId,rating,timestamp,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995,1,4.0,964982703,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),1995,5,4.0,847434962,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),1995,7,4.5,1106635946,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),1995,15,2.5,1510577970,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),1995,17,4.5,1305696483,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,1,Toy Story (1995),1995,18,3.5,1455209816,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,1,Toy Story (1995),1995,19,4.0,965705637,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,1,Toy Story (1995),1995,21,3.5,1407618878,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,1,Toy Story (1995),1995,27,3.0,962685262,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,1,Toy Story (1995),1995,31,5.0,850466616,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## FILTRO BASADO EN POPULARIDAD

### TOP 10 PELICULAS MEJOR CALIFICADAS

In [245]:
##### recomendaciones basado en popularidad ######
mej_cal = pd.read_sql(
    """select title, 
            avg(rating) as avg_rat,
            count(*) as cal_num
            from data_clean
            group by title
            order by avg_rat desc
            limit 10
            """,
    conn,
)
# Redondear 1 decimal a avg_rat
mej_cal["avg_rat"] = mej_cal["avg_rat"].round(1)
mej_cal

,title,avg_rat,cal_num
0,"Shawshank Redemption, The (1994)",4.4,302
1,Lawrence of Arabia (1962),4.3,44
2,"Godfather, The (1972)",4.3,182
3,Dr. Strangelove or: How I Learned to Stop Worr...,4.3,94
4,Patton (1970),4.3,31
5,Cool Hand Luke (1967),4.3,55
6,Chinatown (1974),4.3,57
7,Rear Window (1954),4.3,84
8,Fight Club (1999),4.3,210
9,"Princess Bride, The (1987)",4.3,136


### TOP 10 PELÍCULAS CON MÁS CALIFICACIONES

In [244]:
##### recomendaciones basado en popularidad ######
mej_cal = pd.read_sql(
    """select title, 
            avg(rating) as avg_rat,
            count(*) as cal_num
            from data_clean
            group by title
            order by cal_num desc
            limit 10
            """,
    conn,
)
# Redondear 1 decimal a avg_rat
mej_cal["avg_rat"] = mej_cal["avg_rat"].round(1)
mej_cal

,title,avg_rat,cal_num
0,Forrest Gump (1994),4.2,315
1,"Shawshank Redemption, The (1994)",4.4,302
2,Pulp Fiction (1994),4.2,294
3,"Matrix, The (1999)",4.2,267
4,"Silence of the Lambs, The (1991)",4.2,263
5,Star Wars: Episode IV - A New Hope (1977),4.2,241
6,Jurassic Park (1993),3.7,229
7,Braveheart (1995),4.0,225
8,Terminator 2: Judgment Day (1991),4.0,217
9,Fight Club (1999),4.3,210


### TOP 10 PELÍCULAS MEJOR CALIFICADAS POR AÑO

In [219]:
cal_year = pd.read_sql(
    """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, avg_rat desc
            """,
    conn,
)

# Aproximar a 2 decimales
cal_year = cal_year.round(1)

# Escoger de la tabla anterior 10 peliculas por año con mejor calificación
cal_year = cal_year.groupby("year").head(10)

# Visualizar
cal_year

,year,title,avg_rat,count_rat
0,2016,Zootopia (2016),3.9,32
1,2016,Deadpool (2016),3.8,53
2,2015,The Martian (2015),4.0,48
3,2015,Kingsman: The Secret Service (2015),4.0,36
4,2015,Star Wars: Episode VII - The Force Awakens (2015),3.9,40
...,...,...,...,...
856,1940,Fantasia (1940),3.8,52
857,1940,Pinocchio (1940),3.5,59
858,1939,"Wizard of Oz, The (1939)",3.9,91
859,1939,Gone with the Wind (1939),3.6,45


In [213]:
# Función pidiendo el año
def cal_year(year):
    cal_year = pd.read_sql(
        """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, avg_rat desc
            """,
        conn,
    )

    # Aproximar a 2 decimales
    cal_year = cal_year.round(1)

    # Escoger de la tabla anterior 10 peliculas por año con mejor calificación pero mostrando todos los años
    cal_year = cal_year.groupby("year").head(10)

    return cal_year[cal_year["year"] == year]


cal_year("2003")

,year,title,avg_rat,count_rat
190,2003,"Lord of the Rings: The Return of the King, The...",4.1,182
191,2003,Old Boy (2003),4.1,39
192,2003,Lost in Translation (2003),4.0,74
193,2003,Kill Bill: Vol. 1 (2003),4.0,130
194,2003,Finding Nemo (2003),4.0,139
195,2003,"Last Samurai, The (2003)",3.9,61
196,2003,Big Fish (2003),3.8,68
197,2003,Love Actually (2003),3.8,59
198,2003,Pirates of the Caribbean: The Curse of the Bla...,3.8,148
199,2003,Matchstick Men (2003),3.7,34


### TOP 10 PELICULAS CON MÁS CALIFICACIONES POR AÑO


In [220]:
cal_year = pd.read_sql(
    """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, count_rat desc
            """,
    conn,
)

# Aproximar a 2 decimales
cal_year = cal_year.round(1)

# Escoger de la tabla anterior 10 películas por año con mejor calificación
cal_year = cal_year.groupby("year").head(10)

# Visualizar
cal_year

,year,title,avg_rat,count_rat
0,2016,Deadpool (2016),3.8,53
1,2016,Zootopia (2016),3.9,32
2,2015,The Martian (2015),4.0,48
3,2015,Mad Max: Fury Road (2015),3.8,47
4,2015,Inside Out (2015),3.8,43
...,...,...,...,...
856,1940,Pinocchio (1940),3.5,59
857,1940,Fantasia (1940),3.8,52
858,1939,"Wizard of Oz, The (1939)",3.9,91
859,1939,Gone with the Wind (1939),3.6,45


In [221]:
# Función pidiendo el año
def cal_year(year):
    cal_year = pd.read_sql(
        """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, count_rat desc
            """,
        conn,
    )

    # Aproximar a 2 decimales
    cal_year = cal_year.round(1)

    # Escoger de la tabla anterior 10 películas por año con mejor calificación pero mostrando todos los años
    cal_year = cal_year.groupby("year").head(10)

    return cal_year[cal_year["year"] == year]


cal_year("2003")

,year,title,avg_rat,count_rat
190,2003,"Lord of the Rings: The Return of the King, The...",4.1,182
191,2003,Pirates of the Caribbean: The Curse of the Bla...,3.8,148
192,2003,Finding Nemo (2003),4.0,139
193,2003,Kill Bill: Vol. 1 (2003),4.0,130
194,2003,"Matrix Reloaded, The (2003)",3.3,95
195,2003,"Matrix Revolutions, The (2003)",3.2,79
196,2003,X2: X-Men United (2003),3.7,75
197,2003,Lost in Translation (2003),4.0,74
198,2003,Bruce Almighty (2003),3.3,71
199,2003,Big Fish (2003),3.8,68


### TOP 1 PELÍCULA MEJOR CALIFICADA POR GÉNERO

In [227]:
# Mejores películas por género
# (nota: Film noir y Sci fi nos da problemas)

lista = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    # "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    # "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for i in lista:
    cal_action = pd.read_sql(
        f"""select title, 
                avg(rating) as avg_rat,
                count(*) as count_rat
                from data_clean
                where {i} = 1
                group by  title
                order by avg_rat desc limit 1
                """,
        conn,
    )

    best_avg = cal_action["avg_rat"].tolist()
    best_genres = cal_action["title"].tolist()
    best_count = cal_action["count_rat"].tolist()

    print(
        f"La mejor pelicula de {i} es: {best_genres[0]} con {best_count[0]} votos y {round(best_avg[0],1)} estrellas "
    )

La mejor pelicula de Action es: Fight Club (1999) con 210 votos y 4.3 estrellas 
La mejor pelicula de Adventure es: Lawrence of Arabia (1962) con 44 votos y 4.3 estrellas 
La mejor pelicula de Animation es: Spirited Away (Sen to Chihiro no kamikakushi) (2001) con 86 votos y 4.1 estrellas 
La mejor pelicula de Children es: Toy Story 3 (2010) con 53 votos y 4.1 estrellas 
La mejor pelicula de Comedy es: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) con 94 votos y 4.3 estrellas 
La mejor pelicula de Crime es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Documentary es: Roger & Me (1989) con 30 votos y 3.9 estrellas 
La mejor pelicula de Drama es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Fantasy es: Princess Bride, The (1987) con 136 votos y 4.3 estrellas 
La mejor pelicula de Horror es: Silence of the Lambs, The (1991) con 263 votos y 4.2 estrellas 
La mejor pelicula de IMAX e

### TOP 1 PELÍCULA CON MÁS CALIFICACIONES POR GÉNERO

In [225]:
# Mejores películas por género
# (nota: Film noir y Sci fi nos da problemas)

lista = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    # "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    # "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for i in lista:
    cal_action = pd.read_sql(
        f"""select title, 
                avg(rating) as avg_rat,
                count(*) as count_rat
                from data_clean
                where {i} = 1
                group by  title
                order by count_rat desc limit 1
                """,
        conn,
    )
    best_avg = cal_action["avg_rat"].tolist()
    best_count = cal_action["count_rat"].tolist()
    best_genres = cal_action["title"].tolist()

    print(
        f"La mejor pelicula de {i} es: {best_genres[0]} con {best_count[0]} votos y {round(best_avg[0],1)} estrellas "
    )

La mejor pelicula de Action es: Matrix, The (1999) con 267 votos y 4.2 estrellas 
La mejor pelicula de Adventure es: Star Wars: Episode IV - A New Hope (1977) con 241 votos y 4.2 estrellas 
La mejor pelicula de Animation es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Children es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Comedy es: Forrest Gump (1994) con 315 votos y 4.2 estrellas 
La mejor pelicula de Crime es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Documentary es: Bowling for Columbine (2002) con 58 votos y 3.8 estrellas 
La mejor pelicula de Drama es: Forrest Gump (1994) con 315 votos y 4.2 estrellas 
La mejor pelicula de Fantasy es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Horror es: Silence of the Lambs, The (1991) con 263 votos y 4.2 estrellas 
La mejor pelicula de IMAX es: Apollo 13 (1995) con 190 votos y 3.8 estrellas 
La mejor pelicula de Musical es: A

## FILTRO BASADO EN CONTENIDO (un solo producto manual)

In [3]:
# df quitando los title duplicados (esto se hizo porque se tomó de la base unificada)
movies = pd.read_sql_query("SELECT * FROM data_clean", conn)
movies = movies.drop_duplicates(subset=["title"])
movies = movies.reset_index(drop=True)

from sklearn.preprocessing import MinMaxScaler

# Cambiar el tipo de dato de la columna year
movies["year"] = movies.year.astype("int64")

##### escalar para que año esté en el mismo rango ###
sc = MinMaxScaler()
movies[["year"]] = sc.fit_transform(movies[["year"]])

## eliminar filas que no se van a utilizar ###
movies_dum = movies.drop(columns=["movieId", "title", "userId", "rating", "timestamp"])
movies = movies.drop(columns=["movieId", "userId", "rating", "timestamp"])

#### No es necesario convertir a dummies porque no se tienen más caracteristicas
movies_dum

,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.734177,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0.734177,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.734177,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0.734177,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.734177,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,1.000000,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
857,0.987342,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
858,0.987342,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
859,0.987342,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [36]:
from ipywidgets import interact  # Módulo para crear widgets interactivos


def recomendacion(movie=list(movies["title"].sort_values()), top=10):
    # Buscar el índice de la película
    ind_movie = movies[movies["title"] == movie].index.values.astype(int)[0]
    # Calcular la correlación
    similar_movies = movies_dum.corrwith(movies_dum.iloc[ind_movie, :], axis=1)
    # Ordenar de mayor a menor
    similar_movies = similar_movies.sort_values(ascending=False)
    # Tomar las indicadas quitando la primera (que es la misma película)
    top_similar_movies = similar_movies.to_frame(name="correlación").iloc[1:top,]
    # Agregar el título
    top_similar_movies["title"] = movies["title"]

    return top_similar_movies


print(interact(recomendacion))

interactive(children=(Dropdown(description='movie', options=('(500) Days of Summer (2009)', '10 Things I Hate …

<function recomendacion at 0x00000251EAB909A0>
